In [1]:
# default_exp utils

# Repository Helpers

This notebook includes code for helping populate the README as well as other more general tasks related to keeping the repository tidy

<br>

### Imports

In [2]:
#export
import pandas as pd

from slack_logger import SlackHandler, SlackFormatter
import logging

import dotenv
import os

<br>

### Loading Environment Variables

First we'll load the the environment variables

In [3]:
env_vars_fp = '../.env'
dotenv.load_dotenv(env_vars_fp)

slack_id = os.environ.get('slack_id')
slack_webhook_url = os.environ.get('slack_webhook_url')

<br>

### Notebook Information

In [4]:
#export
def create_markdown_table(table_info: dict, index_name: str='Id') -> str:
    """
    Returns a string for a markdown table, formatted
    according to the dictionary passed as `table_info` 
    
    Parameters:
        table_info: Mapping from index to values
        index_name: Name to use for the index column

    Returns:
        md_str: Markdown formatted table string  

    Example:
        >>> table_info = {
                'Apples': {
                    'Cost': '40p',
                    'Colour': 'Red/green',
                },
                'Oranges': {
                    'Cost': '50p',
                    'Colour': 'Orange',
                },
            }
        >>> md_str = create_markdown_table(table_info, index_name='Fruit')
        >>> print(md_str) 
        | Fruit   | Cost   | Colour    |
        |:--------|:-------|:----------|
        | Apples  | 40p    | Red/green |
        | Oranges | 50p    | Orange    |
    
    """
    
    df_info = pd.DataFrame(table_info).T
    df_info.index.name = index_name
    
    md_str = df_info.to_markdown()
    
    return md_str

<br>

We can now easily construct markdown tables

In [5]:
notebook_info = {
    # development
    'Repository Helpers': {
        'Directory': 'notebooks',
        'Number': '00',
        'Description': 'Code for keeping the repository tidy',
        'Maintainer': 'Ayrton Bourn'
    },
    'EUMETSAT API Wrapper': {
        'Directory': 'notebooks',
        'Number': '01',
        'Description': 'Development of the API wrapper for ems',
        'Maintainer': 'Ayrton Bourn'
    },
    'Data Transformation': {
        'Directory': 'notebooks',
        'Number': '02',
        'Description': 'Intial EDA and transformation comparisons',
        'Maintainer': 'Ayrton Bourn'
    },
    # usage_examples
    'EUMETSAT Download': {
        'Directory': 'usage_examples',
        'Number': '00',
        'Description': 'Guidance for using the ems download manager',
        'Maintainer': 'Ayrton Bourn'
    },
}

nb_table_str = create_markdown_table(notebook_info)

print(nb_table_str)

| Id                   | Directory      |   Number | Description                                 | Maintainer   |
|:---------------------|:---------------|---------:|:--------------------------------------------|:-------------|
| Repository Helpers   | notebooks      |       00 | Code for keeping the repository tidy        | Ayrton Bourn |
| EUMETSAT API Wrapper | notebooks      |       01 | Development of the API wrapper for ems      | Ayrton Bourn |
| Data Transformation  | notebooks      |       02 | Intial EDA and transformation comparisons   | Ayrton Bourn |
| EUMETSAT Download    | usage_examples |       00 | Guidance for using the ems download manager | Ayrton Bourn |


<br>

### Logging

In [6]:
#export
def set_up_logging(name: str, log_dir: str, 
                   main_logging_level: str='DEBUG', 
                   slack_logging_level: str='CRITICAL', 
                   slack_webhook_url: str=None, slack_id: str=None) -> logging.Logger:
    """
    `set_up_logging` initialises and configures a custom
    logger for `satip`. The logging level of the file and 
    Jupyter outputs are specified by `main_logging_level`
    whilst the Slack handler uses `slack_logging_level`.
    
    There are three core ways that logs are broadcasted:
    
    - Logging to a specified file
    - Logging to Jupyter cell outputs
    - Logging to Slack
    
    Note that the value passed for `main_logging_level` 
    and `slack_logging_level` must be one of: 
    
    - 'CRITICAL'
    - 'FATAL'
    - 'ERROR'
    - 'WARNING'
    - 'WARN'
    - 'INFO'
    - 'DEBUG'
    - 'NOTSET'
    
    Parameters:
        name: Name of the logger
        log_dir: directory where the logs will be stored
        main_logging_level: Logging level for file and Jupyter
        slack_logging_level: Logging level for Slack
        slack_webhook_url: Webhook for the log Slack channel
        slack_id: Option user-id to mention in Slack

    Returns:
        logger: Custom satip logger

    Example:
        Here we'll create a custom logger that saves data
        to the file 'test_log.txt' and also sends Slack
        messages to the specified user and channel.

        >>> from satip.utils import set_up_logging
        >>> import logging
        >>> logger = set_up_logging('test_log', 
                                    'test_log.txt', 
                                    slack_id=slack_id,
                                    slack_webhook_url=slack_webhook_url)
        >>> logger.log(logging.INFO, 'This will output to file and Jupyter but not to Slack as it is not critical')
        '2020-10-20 10:24:35,367 - INFO - This will output to file and Jupyter but not to Slack as it is not critical'
    
    """
    
    # Initialising logger
    logger = logging.getLogger(name)
    
    # Configuring log level
    logging_levels = [
        'CRITICAL',
        'FATAL',
        'ERROR',
        'WARNING',
        'WARN',
        'INFO',
        'DEBUG',
        'NOTSET'
    ]
    
    assert main_logging_level in logging_levels, f"main_logging_level must be one of {', '.join(logging_levels)}"
    assert slack_logging_level in logging_levels, f"slack_logging_level must be one of {', '.join(logging_levels)}"
    
    logger.setLevel(getattr(logging, main_logging_level))
    
    # Defining global formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    
    # Configuring Jupyter output handler
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    # Configuring file output handler
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    log_fp = f'{log_dir}/{name}.txt'
    file_handler = logging.FileHandler(log_fp, mode='a')
    file_handler.setFormatter(formatter)
    file_handler.setLevel(getattr(logging, main_logging_level))
    logger.addHandler(file_handler)

    # Configuring slack output handler
    if slack_webhook_url is not None:
        slack_handler = SlackHandler(username='logger', url=slack_webhook_url, mention=slack_id)
        slack_handler.setFormatter(SlackFormatter())
        slack_handler.setLevel(getattr(logging, slack_logging_level))
        logger.addHandler(slack_handler)
    
    return logger

<br>

We'll now initialise the logger and make a test log

In [7]:
logger = set_up_logging(__name__, 
                        '.', 
                        slack_id=slack_id,
                        slack_webhook_url=slack_webhook_url)

logger.log(logging.INFO, 'This will output to file and Jupyter but not to Slack as it is not critical')

2020-11-12 09:58:41,301 - INFO - This will output to file and Jupyter but not to Slack as it is not critical


<br>

We'll now shutdown the logger handlers and then delete the log file we just made

In [8]:
handlers = logger.handlers[:]

for handler in handlers:
    handler.close()
    logger.removeHandler(handler)

os.remove(f'{__name__}.txt')

<br>

Finally we'll export the specified functions to the utils.py module

In [10]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_eumetsat.ipynb.
Converted 02_reproj.ipynb.
Converted 03_zarr.ipynb.
